In [1]:
from mp_api.client.mprester import MPRester

In [25]:
from pymatgen.core.periodic_table import Element
import pandas as pd
import numpy as np
import os

In [3]:
API_KEY = "fqPPo7Czb5mkbFh8mltlZd0I33csuKv0"
mpr = MPRester(API_KEY)

# 从mateirial project下载数据

In [50]:
#下载所有含有Li，O的结构
docs=mpr.summary.search(elements=['Li', 'O'], all_fields=False, fields=['material_id', 'composition_reduced', 'formula_pretty','band_gap', 'symmetry', 'chemsys', 'structure'])

Retrieving SummaryDoc documents:   0%|          | 0/17066 [00:00<?, ?it/s]

# 函数

In [57]:
ele_list = docs[0].chemsys.split('-')
# a = ["O", "C"]
def if_tm(element_list):
    # 返回体系中是否有过渡金属,输入应尬是一个列表，里面是各个元素的字符串
    for ele in element_list:
        if 'd' in Element(ele).electronic_structure:
            tm = 1
            break
        else:
            tm = 0
    return bool(tm)

def tm_number(composition):
    #返回结构中过渡金属的化学计量，输入应为上面爬的数据中的composition_reduced属性
    #前置库：pymatgen.periodic_table.Element
    tm = []
    ele_dict= composition.as_dict()
    ele = list(ele_dict.keys())
    for i in ele: 
        if 'd' in Element(i).electronic_structure:
            tm.append(i)
    stoihiometry_tm = 0
    for ele in tm:
        stoihiometry_tm += int(ele_dict[ele])
    return stoihiometry_tm



In [58]:
#选择具有过渡金属且锂原子更多的结构（c2/m）
rich_lithium_docs_c2m = []
for i in docs:
    ele_list = i.chemsys.split('-')
    stoichiometry_li = int(i.composition_reduced.as_dict()["Li"])
    stoichiometry_tm = tm_number(i.composition_reduced)
    crystral_system = i.symmetry.symbol
    if if_tm(ele_list)==True and stoichiometry_li > stoichiometry_tm and crystral_system =="C2/m":
        rich_lithium_docs_c2m.append(i)

In [59]:
df = pd.DataFrame(columns=['formula', 'mp_id', 'crystal system', 'band_gap'])
for i in rich_lithium_docs_c2m:
    data=[i.formula_pretty, i.material_id.string, i.symmetry.symbol, i.band_gap]
    df.loc[len(df)]=data

In [60]:
df

,formula,mp_id,crystal system,band_gap
0,Li5Mn3O8,mp-1174546,C2/m,0.0000
1,Li3MnP2H2O9,mp-780319,C2/m,0.8976
2,Li5Mn(CoO4)2,mp-1174160,C2/m,0.0000
3,Li9Mn2Co5O16,mp-1176312,C2/m,0.0000
4,Li7Mn2(CoO4)3,mp-1174510,C2/m,0.0000
...,...,...,...,...
393,Li3MnCoO5,mp-1174029,C2/m,1.0549
394,Li9Mn2Co5O16,mp-1100747,C2/m,0.0000
395,Li3V2(O2F)2,mp-760200,C2/m,0.0000
396,Li2RuO3,mp-4630,C2/m,0.0000


In [124]:
df_gap = df[['mp_id','band_gap']]



In [82]:

a = pd.cut(test, [-1, 0,10 ], labels=['q','w'])
a

0     q
1     w
2     q
3     q
4     q
5     w
6     q
7     w
8     w
9     w
10    w
Name: band_gap, dtype: category
Categories (2, object): ['q' < 'w']

In [48]:
df_gap.to_csv("c2m.csv", index=False, header=False)

In [125]:
# 离散化数据, 0代表金属导体，1代表半导体，2代表
df_discrete = df_gap
data_discrete = pd.cut(df_discrete['band_gap'], [-1, 0, 8], labels=['0', '1'])
df_discrete['band_gap']= data_discrete


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [126]:
df_discrete.to_csv("c2m_discrete.csv", index=False, header=False)

In [61]:
# 把富锂材料结构存为cif形式
for struct in rich_lithium_docs_c2m:
    mp_id = struct.material_id.string.split('-')[1]
    struct.structure.to('data_c2m/'+mp_id+'.cif')

In [127]:
from matminer.datasets import load_dataset

In [138]:
from matminer.featurizers.composition import ElementProperty
from matminer.featurizers.structure import GlobalSymmetryFeatures
from matminer.featurizers.composition import Meredig